# SwellSight Real-to-Synthetic Pipeline - Depth-Anything-V2 Depth Extraction

This notebook implements the Depth-Anything-V2 depth estimation phase of the SwellSight real-to-synthetic generation pipeline.

## Overview
This notebook provides:
- Depth-Anything-V2 model initialization and configuration
- Batch depth map extraction from real beach images
- Quality assessment and filtering of depth maps
- Depth map visualization and analysis
- Storage and metadata management
- Advanced error handling and memory management

## Pipeline Integration
This notebook implements Step 1 of the pipeline:
1. **Model Loading**: Initialize Depth-Anything-V2 depth estimation model
2. **Batch Processing**: Extract depth maps from all valid images
3. **Quality Control**: Apply quality thresholds and filtering
4. **Storage**: Save depth maps with structured metadata

## Depth-Anything-V2 Models Available
- `depth-anything/Depth-Anything-V2-Large`: Best quality, higher memory usage
- `depth-anything/Depth-Anything-V2-Base`: Balanced quality and speed
- `depth-anything/Depth-Anything-V2-Small`: Fastest, lower memory usage

## Prerequisites
- Complete execution of `02_Data_Import_and_Preprocessing.ipynb`
- Valid images available in processing batch
- Sufficient GPU memory (8GB+ recommended for Large model)

---

## 1. Load Configuration and Processing Batch

In [ ]:
import sys
import os
from pathlib import Path

# Add utils to path for shared utilities
sys.path.append(str(Path.cwd() / 'utils'))

# Import shared utilities
from config_manager import load_config
from data_flow_manager import load_previous_results, save_stage_results
from error_handler import retry_with_backoff, handle_gpu_memory_error
from memory_optimizer import get_optimal_batch_size, cleanup_variables
from progress_tracker import create_progress_bar
from data_validator import validate_image_quality

# Check if running in Google Colab
if 'google.colab' in sys.modules:
    from google.colab import drive
    print("Mounting Google Drive...")

    try:
        # Attempt 1: Standard mount
        drive.mount('/content/drive')
        print("✓ Google Drive mounted successfully")
    except Exception as e:
        print(f"Standard mount failed: {e}")

        # Attempt 2: Force remount with extended timeout (robust fallback)
        print("Trying force remount with extended timeout...")
        try:
            drive.mount('/content/drive', force_remount=True, timeout_ms=300000)
            print("✓ Force remount successful")
        except Exception as e2:
            print(f"❌ Critical failure mounting drive: {e2}")
            raise

    # Verify the specific project path exists
    PROJECT_PATH = Path('/content/drive/MyDrive/SwellSight')
    if PROJECT_PATH.exists():
        print(f"✓ Project directory found: {PROJECT_PATH}")
    else:
        print(f"⚠️ Project directory not found at: {PROJECT_PATH}")
else:
    PROJECT_PATH = Path.cwd()
    print(f"Not running in Google Colab. Using current directory: {PROJECT_PATH}")

In [ ]:
import json
import logging
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm.auto import tqdm
import torch
import warnings
warnings.filterwarnings('ignore')

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

print("🔄 Loading configuration and processing batch...")

try:
    # Load pipeline configuration using shared utility
    PIPELINE_CONFIG = load_config(PROJECT_PATH / 'config.json')
    
    # Load processing batch from previous stage
    PROCESSING_BATCH = load_previous_results(
        stage_name="data_preprocessing",
        required_files=["processed_images.json", "quality_report.json"]
    )
    
    print("✅ Configuration and batch loaded successfully")
    
except Exception as e:
    print(f"❌ Failed to load configuration or batch: {e}")
    print("Please ensure you have run the previous notebooks and have valid configuration.")
    sys.exit(1)

# Display key information
print(f"\n📋 Pipeline Configuration:")
pipeline_name = PIPELINE_CONFIG.get('pipeline', {}).get('name', 'SwellSight Pipeline')
default_depth_model = PIPELINE_CONFIG.get('models', {}).get('depth_model', 'depth-anything/Depth-Anything-V2-Large')
quality_threshold = PIPELINE_CONFIG.get('processing', {}).get('quality_threshold', 0.7)

print(f"   Pipeline: {pipeline_name}")
print(f"   Default Depth model: {default_depth_model}")
print(f"   Quality threshold: {quality_threshold}")

print(f"\n📊 Processing Batch:")
total_images = len(PROCESSING_BATCH.get('processed_images', []))
print(f"   Total images: {total_images}")
print(f"   Ready for processing: {total_images > 0}")

# Set up paths from config
DEPTH_OUTPUT_PATH = Path(PIPELINE_CONFIG['paths']['output_dir']) / 'depth_maps'
METADATA_PATH = Path(PIPELINE_CONFIG['paths']['output_dir']) / 'metadata'
LOGS_PATH = Path(PIPELINE_CONFIG['paths']['output_dir']) / 'logs'

# Ensure directories exist
DEPTH_OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
METADATA_PATH.mkdir(parents=True, exist_ok=True)
LOGS_PATH.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Working directories:")
print(f"   Depth output: {DEPTH_OUTPUT_PATH}")
print(f"   Metadata: {METADATA_PATH}")
print(f"   Logs: {LOGS_PATH}")

## 2. Device Configuration and GPU Setup

In [ ]:
# Configure device for Depth-Anything-V2 processing
print("🔧 Configuring device for Depth-Anything-V2 processing...")

# Device detection with memory optimization
if torch.cuda.is_available():
    device = "cuda"
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9

    print(f"\n🚀 GPU Configuration:")
    print(f"   Device: {device}")
    print(f"   GPU: {gpu_name}")
    print(f"   Memory: {gpu_memory:.1f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")

    # Memory recommendations for Depth-Anything-V2
    if gpu_memory < 6:
        print("   ⚠️  Warning: Less than 6GB GPU memory. Consider using Small model.")
        recommended_model = "depth-anything/Depth-Anything-V2-Small"
    elif gpu_memory < 12:
        print("   ✅ Good: Sufficient memory for Base model.")
        recommended_model = "depth-anything/Depth-Anything-V2-Base"
    else:
        print("   ✅ Excellent: Sufficient memory for Large model.")
        recommended_model = "depth-anything/Depth-Anything-V2-Large"

else:
    device = "cpu"
    gpu_memory = 0
    gpu_name = "CPU"
    recommended_model = "depth-anything/Depth-Anything-V2-Small"  # Fastest for CPU
    print(f"\n⚠️  CPU Configuration:")
    print(f"   Device: {device}")
    print(f"   Recommended model: {recommended_model} (fastest for CPU)")
    print(f"   Note: CPU processing will be significantly slower")

# Model selection - use recommended based on hardware
selected_model = recommended_model

print(f"\n🧠 Model Selection:")
print(f"   Default from config: {default_depth_model}")
print(f"   Recommended for hardware: {recommended_model}")
print(f"   Selected: {selected_model}")

if selected_model != default_depth_model:
    print(f"   ℹ️  Using hardware-optimized model instead of default")

# Store device configuration
DEVICE_CONFIG = {
    'device': device,
    'selected_model': selected_model,
    'gpu_memory_gb': gpu_memory,
    'gpu_name': gpu_name
}

print(f"\n✅ Device configuration completed!")

## 3. Depth-Anything-V2 Model Initialization

In [ ]:
import cv2
import numpy as np
from dataclasses import dataclass
from transformers import pipeline
import torch.nn.functional as F

@dataclass
class DepthResult:
    """Data class to store depth extraction results."""
    depth_map: np.ndarray
    depth_quality_score: float
    processing_time: float
    model_used: str

class DepthAnythingV2Extractor:
    """Depth-Anything-V2 depth extractor with memory optimization and error handling."""

    def __init__(self, model_name="depth-anything/Depth-Anything-V2-Large", device="cuda", storage_path=None):
        self.device = device
        self.storage_path = Path(storage_path) if storage_path else None
        self.model_name = model_name
        self.pipe = None
        
        print(f"   Loading Depth-Anything-V2 model: {model_name} on {device}...")
        
        try:
            # Initialize the depth estimation pipeline
            self.pipe = pipeline(
                task="depth-estimation",
                model=model_name,
                device=0 if device == "cuda" else -1,
                torch_dtype=torch.float16 if device == "cuda" else torch.float32
            )
            print(f"   Model loaded successfully: {model_name}")
            
        except Exception as e:
            print(f"❌ Error loading model: {e}")
            raise

    def _calculate_quality_score(self, depth_map):
        """Calculate depth map quality score based on statistical measures."""
        try:
            # Normalize depth map
            depth_norm = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min() + 1e-8)
            
            # Calculate various quality metrics
            std_score = np.std(depth_norm)  # Higher std indicates more depth variation
            gradient_score = np.mean(np.abs(np.gradient(depth_norm)))  # Edge information
            
            # Combine metrics (weighted average)
            quality_score = 0.6 * std_score + 0.4 * gradient_score
            
            # Clamp to [0, 1] range
            return min(1.0, max(0.0, quality_score))
            
        except Exception as e:
            logger.warning(f"Quality calculation failed: {e}")
            return 0.5  # Default moderate quality

    def extract_depth(self, image_path, store_result=True):
        """Extract depth map from an image with error handling."""
        start_time = datetime.now()
        
        try:
            # Load and validate image
            image = Image.open(image_path).convert('RGB')
            
            # Validate image quality first
            quality_check = validate_image_quality(str(image_path))
            if not quality_check['is_valid']:
                raise ValueError(f"Image quality validation failed: {quality_check['issues']}")
            
            # Extract depth using pipeline
            with torch.no_grad():
                result = self.pipe(image)
                depth_map = np.array(result['depth'])
            
            # Calculate processing time
            processing_time = (datetime.now() - start_time).total_seconds()
            
            # Calculate quality score
            quality_score = self._calculate_quality_score(depth_map)
            
            # Store result if requested
            if store_result and self.storage_path:
                self.storage_path.mkdir(parents=True, exist_ok=True)
                save_path = self.storage_path / f"{Path(image_path).stem}_depth.npy"
                np.save(save_path, depth_map)
            
            return DepthResult(
                depth_map=depth_map,
                depth_quality_score=quality_score,
                processing_time=processing_time,
                model_used=self.model_name
            )
            
        except Exception as e:
            # Log individual image failure but don't stop batch processing
            logger.error(f"Failed to process {image_path}: {e}")
            raise e

    def cleanup(self):
        """Clean up model resources."""
        if self.pipe is not None:
            del self.pipe
            self.pipe = None
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

# Initialize depth extractor with error handling and GPU fallback
print("🧠 Initializing Depth-Anything-V2 depth extractor...")

try:
    print(f"\n📦 Initializing Depth-Anything-V2 with:")
    print(f"   Model: {selected_model}")
    print(f"   Device: {device}")
    print(f"   Storage path: {DEPTH_OUTPUT_PATH}")

    # Initialize depth extractor
    depth_extractor = DepthAnythingV2Extractor(
        model_name=selected_model,
        device=device,
        storage_path=str(DEPTH_OUTPUT_PATH)
    )

    print(f"✅ Depth-Anything-V2 extractor initialized successfully!")
    print(f"   Ready for batch processing")

except Exception as e:
    print(f"❌ Failed to initialize Depth-Anything-V2 extractor: {e}")

    # GPU fallback mechanism
    if device == "cuda":
        print("\n🔄 Attempting fallback to CPU...")
        try:
            depth_extractor = DepthAnythingV2Extractor(
                model_name="depth-anything/Depth-Anything-V2-Small",
                device="cpu",
                storage_path=str(DEPTH_OUTPUT_PATH)
            )
            print("✅ Fallback to CPU successful!")
            DEVICE_CONFIG['device'] = 'cpu'
            DEVICE_CONFIG['selected_model'] = 'depth-anything/Depth-Anything-V2-Small'
        except Exception as fallback_error:
            print(f"❌ CPU fallback also failed: {fallback_error}")
            raise
    else:
        raise

## 4. Batch Depth Extraction Processing

In [ ]:
import time
from collections import defaultdict

print("🔄 Starting batch depth extraction processing...")

# Get list of images to process
images_to_process = PROCESSING_BATCH.get('processed_images', [])
total_images = len(images_to_process)

if total_images == 0:
    print("❌ No images found to process. Please run previous notebooks first.")
    sys.exit(1)

print(f"\n📊 Processing Configuration:")
print(f"   Total images: {total_images}")
print(f"   Quality threshold: {quality_threshold}")
print(f"   Model: {selected_model}")
print(f"   Device: {device}")

# Calculate optimal batch size for memory management
if device == "cuda":
    # Estimate memory per image (conservative estimate)
    estimated_memory_per_image = 0.5  # GB
    optimal_batch_size = get_optimal_batch_size(
        available_memory=gpu_memory * 0.8,  # Use 80% of available memory
        item_size=estimated_memory_per_image
    )
else:
    optimal_batch_size = 1  # Process one at a time on CPU

print(f"   Optimal batch size: {optimal_batch_size}")

# Initialize tracking variables
successful_results = []
failed_results = []
processing_times = []
quality_scores = []
error_counts = defaultdict(int)

print(f"\n🚀 Starting depth extraction...")
print(f"   Progress will be shown below")

# Process images with progress tracking and error handling
with create_progress_bar(total_images, "Extracting depth maps") as pbar:
    for i, image_info in enumerate(images_to_process):
        try:
            image_path = image_info.get('path') or image_info.get('file_path')
            if not image_path:
                raise ValueError("No image path found in batch data")
            
            # Convert to Path object if string
            if isinstance(image_path, str):
                image_path = Path(image_path)
            
            # Check if image exists
            if not image_path.exists():
                raise FileNotFoundError(f"Image not found: {image_path}")
            
            # Extract depth with retry logic
            def extract_with_retry():
                return depth_extractor.extract_depth(image_path, store_result=True)
            
            result = retry_with_backoff(
                func=extract_with_retry,
                max_retries=3,
                backoff_factor=2.0
            )
            
            # Track successful result
            successful_results.append({
                'image_path': str(image_path),
                'depth_quality_score': result.depth_quality_score,
                'processing_time': result.processing_time,
                'model_used': result.model_used,
                'depth_shape': result.depth_map.shape
            })
            
            processing_times.append(result.processing_time)
            quality_scores.append(result.depth_quality_score)
            
        except torch.cuda.OutOfMemoryError as e:
            # Handle GPU memory errors with fallback
            error_msg = f"GPU memory error for {image_path}: {e}"
            logger.error(error_msg)
            
            try:
                # Try CPU fallback for this image
                result = handle_gpu_memory_error(
                    operation="depth_extraction",
                    fallback_func=lambda: depth_extractor.extract_depth(image_path, store_result=True)
                )
                
                successful_results.append({
                    'image_path': str(image_path),
                    'depth_quality_score': result.depth_quality_score,
                    'processing_time': result.processing_time,
                    'model_used': result.model_used + " (CPU fallback)",
                    'depth_shape': result.depth_map.shape
                })
                
            except Exception as fallback_error:
                failed_results.append({
                    'image_path': str(image_path),
                    'error': str(fallback_error),
                    'error_type': 'gpu_memory_fallback_failed'
                })
                error_counts['gpu_memory_fallback_failed'] += 1
            
        except Exception as e:
            # Handle other errors - log and continue
            error_type = type(e).__name__
            error_msg = f"Failed to process {image_path}: {e}"
            logger.error(error_msg)
            
            failed_results.append({
                'image_path': str(image_path),
                'error': str(e),
                'error_type': error_type
            })
            error_counts[error_type] += 1
        
        # Update progress
        pbar.update(1)
        
        # Memory cleanup every 10 images
        if (i + 1) % 10 == 0:
            cleanup_variables(['result'])
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

# Calculate statistics
successful_extractions = len(successful_results)
failed_extractions = len(failed_results)
success_rate = (successful_extractions / total_images) * 100 if total_images > 0 else 0

# Quality threshold analysis
above_threshold = sum(1 for score in quality_scores if score >= quality_threshold)
below_threshold = len(quality_scores) - above_threshold

print(f"\n✅ Batch depth extraction completed!")
print(f"\n📊 Processing Results:")
print(f"   Total images processed: {total_images}")
print(f"   Successful extractions: {successful_extractions}")
print(f"   Failed extractions: {failed_extractions}")
print(f"   Low quality (below threshold): {below_threshold}")
print(f"   Success rate: {success_rate:.1f}%")

if processing_times:
    print(f"\n⏱️  Processing Performance:")
    print(f"   Average time per image: {np.mean(processing_times):.2f} seconds")
    print(f"   Total processing time: {sum(processing_times):.1f} seconds")
    print(f"   Fastest: {min(processing_times):.2f}s, Slowest: {max(processing_times):.2f}s")

if error_counts:
    print(f"\n❌ Error Summary:")
    for error_type, count in error_counts.items():
        print(f"   {error_type}: {count} occurrences")

## 5. Save Results and Generate Reports

In [ ]:
print("💾 Saving depth extraction results and generating reports...")

# Prepare comprehensive results
depth_extraction_results = {
    'stage_info': {
        'stage_name': 'depth_anything_v2_extraction',
        'timestamp': datetime.now().isoformat(),
        'model_used': selected_model,
        'device_used': device,
        'total_processing_time': sum(processing_times) if processing_times else 0
    },
    'processing_statistics': {
        'total_images': total_images,
        'successful_extractions': successful_extractions,
        'failed_extractions': failed_extractions,
        'success_rate': success_rate,
        'above_quality_threshold': above_threshold,
        'below_quality_threshold': below_threshold
    },
    'performance_metrics': {
        'mean_processing_time': np.mean(processing_times) if processing_times else 0,
        'std_processing_time': np.std(processing_times) if processing_times else 0,
        'min_processing_time': min(processing_times) if processing_times else 0,
        'max_processing_time': max(processing_times) if processing_times else 0,
        'mean_quality_score': np.mean(quality_scores) if quality_scores else 0,
        'std_quality_score': np.std(quality_scores) if quality_scores else 0
    },
    'successful_results': successful_results,
    'failed_results': failed_results,
    'error_summary': dict(error_counts),
    'configuration': {
        'quality_threshold': quality_threshold,
        'batch_size': optimal_batch_size,
        'device_config': DEVICE_CONFIG
    }
}

# Save results using shared utility
try:
    save_stage_results(
        data=depth_extraction_results,
        stage_name="depth_anything_v2_extraction",
        metadata={
            'model_used': selected_model,
            'processing_time': sum(processing_times) if processing_times else 0,
            'success_rate': success_rate
        }
    )
    
    print("✅ Results saved successfully!")
    
except Exception as e:
    print(f"⚠️ Warning: Failed to save results: {e}")
    # Continue execution even if saving fails

# Generate quality report
quality_report = {
    'timestamp': datetime.now().isoformat(),
    'total_depth_maps': successful_extractions,
    'quality_threshold': quality_threshold,
    'quality_distribution': {
        'above_threshold': above_threshold,
        'below_threshold': below_threshold,
        'percentage_above': (above_threshold / successful_extractions * 100) if successful_extractions > 0 else 0
    },
    'quality_statistics': {
        'mean': np.mean(quality_scores) if quality_scores else 0,
        'median': np.median(quality_scores) if quality_scores else 0,
        'std': np.std(quality_scores) if quality_scores else 0,
        'min': min(quality_scores) if quality_scores else 0,
        'max': max(quality_scores) if quality_scores else 0
    }
}

print(f"\n📋 Quality Report Summary:")
print(f"   Total depth maps generated: {successful_extractions}")
print(f"   Above quality threshold ({quality_threshold}): {above_threshold} ({quality_report['quality_distribution']['percentage_above']:.1f}%)")
print(f"   Mean quality score: {quality_report['quality_statistics']['mean']:.3f}")
print(f"   Quality score range: {quality_report['quality_statistics']['min']:.3f} - {quality_report['quality_statistics']['max']:.3f}")

print(f"\n✅ Depth extraction stage completed successfully!")